In [101]:
from sentence_transformers import SentenceTransformer
import os

# Попробуйте увеличить таймаут (в секундах)
os.environ['HF_TRANSFER_TIMEOUT'] = '100'  # Увеличиваем таймаут для HF
os.environ['HF_HUB_DOWNLOAD_TIMEOUT'] = '600'  # Увеличиваем таймаут загрузки

embedding_model = SentenceTransformer(
    model_name_or_path='sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
    # Дополнительные опции для requests
    request_timeout=120
)
test_embedding = embedding_model.embed_query("FastAPI is a modern web frameworffk")
print(f"Размерность эмбеддинга: {len(test_embedding)}")

/home/vetak/projects/envs/env_3.12.3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: SentenceTransformer.__init__() got an unexpected keyword argument 'request_timeout'

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
# Инициализация модели для создания эмбеддингов
# Хорошая модель для старта: all-MiniLM-L6-v2
embedding_model = HuggingFaceEmbeddings(
    # model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_name = 'intfloat/multilingual-e5-large',
    model_kwargs={'device': 'cpu'}, 
    encode_kwargs={'normalize_embeddings': True}  # Нормализация для косинусного сходства
)

# Тестируем модель
test_embedding = embedding_model.embed_query("FastAPI is a modern web frameworffk")
print(f"Размерность эмбеддинга: {len(test_embedding)}")

TypeError: SentenceTransformer.__init__() got an unexpected keyword argument 'model_name'

In [7]:
client = QdrantClient(url='localhost:6333')  # Используйте "localhost:6333" для standalone-сервера

In [5]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from langchain.vectorstores import Qdrant

# Локальный режим (для тестирования)
client = QdrantClient(":memory:")  # Используйте "localhost:6333" для standalone-сервера

# Или подключение к Qdrant Cloud
# client = QdrantClient(
#     url="your-cluster-url",
#     api_key="your-api-key",
# )

# Создание коллекции (если её нет)
try:
    client.create_collection(
        collection_name="fastapi_docs",
        vectors_config=VectorParams(
            size=384,  # Должно соответствовать размерности выбранной модели эмбеддингов!
            distance=Distance.COSINE  # Косинусное расстояние для семантического сходства
        )
    )
except Exception as e:
    print(f"Коллекция可能 уже существует: {e}")

# Инициализация векторного хранилища LangChain для Qdrant
vector_store = Qdrant(
    client=client,
    collection_name="fastapi_docs",
    embeddings=embedding_model,
)

NameError: name 'embedding_model' is not defined

In [4]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from langchain.vectorstores import Qdrant

# Локальный режим (для тестирования)
client = QdrantClient(":memory:")  # Используйте "localhost:6333" для standalone-сервера

# Или подключение к Qdrant Cloud
# client = QdrantClient(
#     url="your-cluster-url",
#     api_key="your-api-key",
# )

# Создание коллекции (если её нет)
try:
    client.create_collection(
        collection_name="fastapi_docs",
        vectors_config=VectorParams(
            size=384,  # Должно соответствовать размерности выбранной модели эмбеддингов!
            distance=Distance.COSINE  # Косинусное расстояние для семантического сходства
        )
    )
except Exception as e:
    print(f"Коллекция可能 уже существует: {e}")

# Инициализация векторного хранилища LangChain для Qdrant
vector_store = Qdrant(
    client=client,
    collection_name="fastapi_docs",
    embeddings=embedding_model,
)

NameError: name 'embedding_model' is not defined

In [ ]:
# Пример запроса
query = "How to create a POST endpoint in FastAPI?"
found_docs = vector_store.similarity_search(
    query=query,
    k=5  # Количество возвращаемых результатов
)

# Вывод результатов
for i, doc in enumerate(found_docs):
    print(f"Результат #{i+1}:")
    print(f"Контент: {doc.page_content[:200]}...")
    print(f"Источник: {doc.metadata.get('source', 'N/A')}")
    print(f"Рейтинг: {score}")  # Если используется similarity_search_with_score
    print("-" * 50)

In [17]:
import os
from langchain.docstore.document import  Document
from langchain.text_splitter import MarkdownHeaderTextSplitter
from tqdm import tqdm
from typing import List, Tuple
PATH = './docs/'

def get_chunks(path) -> List[Tuple[str, str]]:
    headers_to_split_on = [
        ('#', 'Header 1'),
        ('##', 'Header 2'),
        ('###', 'Header 3'),
        # ('###', 'Header 4')
    ]

    all_chunks = []
    for root, _, files in os.walk(path):
        for file in tqdm(files):
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                try:
                    # Загрузка файла
                    with open(file_path) as f:
                        markdown_content = f.read()
                        
                    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
                    doc_chunks = markdown_splitter.split_text(markdown_content)
                    
                    for chunk in doc_chunks:
                        all_chunks.append((list(chunk.metadata.values())[0], chunk.page_content)) # на выходе список кортежей вида (заголовок, текст)
                   
                except Exception as e:
                    print(f'Ошибка обработки файла {file_path}: {e}')
                    continue

    print(f'Всего создано {len(all_chunks)} чанков из {len(files)} файлов')
    return all_chunks
                

# def chunk_cleaner(chunk):
    

In [18]:
c = get_chunks(PATH)

100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 99.01it/s]

Всего создано 709 чанков из 5 файлов


In [20]:
c[:3]

[('Альтернативы, источники вдохновения и сравнения',
  '# Альтернативы, источники вдохновения и сравнения  \nЧто вдохновило на создание **FastAPI**, сравнение его с альтернативами и чему он научился у них.'),
 ('Альтернативы, источники вдохновения и сравнения',
  '## Введение  \n**FastAPI** не существовал бы, если б не было более ранних работ других людей.  \nОни создали большое количество инструментов, которые вдохновили меня на создание **FastAPI**.  \nЯ всячески избегал создания нового фреймворка в течение нескольких лет.\nСначала я пытался собрать все нужные функции, которые ныне есть в **FastAPI**, используя множество различных фреймворков, плагинов и инструментов.  \nНо в какой-то момент не осталось другого выбора, кроме как создать что-то, что предоставляло бы все эти функции сразу.\nВзять самые лучшие идеи из предыдущих инструментов и, используя новые возможности Python (которых не было до версии 3.6, то есть подсказки типов), объединить их.'),
 ('Альтернативы, источники вдохно

In [34]:
import re
def clean_text(text: str) -> str:
    '''Чистка текста от HTML-тегов, лишних символов и сусора с сохранением структуры и содержания'''

    clean_text = text.strip() 
    # Удаляем теги HTML-теги
    clean_text = re.sub(r'<a\s+href="([^"]*)"[^>]*>([^<]*)</a>', r'\2 (\1)', clean_text)
    clean_text = re.sub(r'///\s*\w+\s*|.*?///', '', clean_text, re.DOTALL) # Символы типа '///'
    clean_text = re.sub(r'#+\s*', '', clean_text)  # Заголовки
    return clean_text

In [33]:
t = '### <a href="https://www.django-rest-framework.org/" class="external-link" target="_blank">Django REST Framework</a>  \nФреймворк Django REST был создан, как гибкий инструментарий для создания веб-API на основе Django.  \nDRF использовался многими компаниями, включая Mozilla, Red Hat и Eventbrite.  \nЭто был один из первых примеров **автоматического документирования API** и это была одна из первых идей, вдохновивших на создание **FastAPI**.  \n/// note | Заметка  \nDjango REST Framework был создан Tom Christie.\nОн же создал Starlette и Uvicorn, на которых основан **FastAPI**.  \n///  \n/// check | Идея для **FastAPI**  \nДолжно быть автоматическое создание документации API с пользовательским веб-интерфейсом.  \n///'
clean_text(t)

'Django REST Framework (https://www.django-rest-framework.org/)  \nФреймворк Django REST был создан, как гибкий инструментарий для создания веб-API на основе Django.  \nDRF использовался многими компаниями, включая Mozilla, Red Hat и Eventbrite.  \nЭто был один из первых примеров **автоматического документирования API** и это была одна из первых идей, вдохновивших на создание **FastAPI**.  \n| Заметка  \nDjango REST Framework был создан Tom Christie.\nОн же создал Starlette и Uvicorn, на которых основан **FastAPI**.  \n  \n| Идея для **FastAPI**  \nДолжно быть автоматическое создание документации API с пользовательским веб-интерфейсом.  \n'

In [93]:
import os
import re
from langchain.docstore.document import  Document
from langchain.text_splitter import MarkdownHeaderTextSplitter
from tqdm import tqdm
from typing import List, Tuple
PATH = './docs/'

def get_chunks(path) -> List[Tuple[str, str]]:
    headers_to_split_on = [
        ('#', 'Header 1'),
        ('##', 'Header 2'),
        ('###', 'Header 3'),
    ]

    all_chunks = []
    for root, _, files in os.walk(path):
        for file in tqdm(files):
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                try:
                    # Загрузка файла
                    with open(file_path, 'r', encoding='utf-8') as f:
                        markdown_content = f.read()
                        
                    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
                    doc_chunks = markdown_splitter.split_text(markdown_content)
                    
                    for chunk in doc_chunks:                        
                        content = clean_text(chunk.page_content) # Чистим текст
                        all_chunks.append((list(chunk.metadata.values())[0], content)) # на выходе список кортежей вида (заголовок, текст)
                   
                except Exception as e:
                    print(f'Ошибка обработки файла {file_path}: {e}')
                    continue

    print(f'\nВсего создано {len(all_chunks)} чанков из {len(files)} файлов')
    return all_chunks
                


def clean_text(text: str) -> str:
    '''Чистка текста от HTML-тегов, лишних символов и сусора с сохранением структуры и содержания'''
    
    if not text or not text.strip():
        return ""

    cleaned = text.strip() 

    # Удаляем теги HTML-теги
    cleaned = re.sub(r'<a\s+href="([^"]*)"[^>]*>([^<]*)</a>', r'\2 (\1)', cleaned)
    cleaned = re.sub(r'<[^>]+>', '', cleaned)

    # Символы типа '///'
    cleaned = re.sub(r'///.*?///', '', cleaned, flags=re.DOTALL)

    # Заголовки
    cleaned = re.sub(r'#+\s*', '', cleaned)
    
    # Жирный текст
    cleaned = re.sub(r'\*\*(.*?)\*\*', r'\1', cleaned) 
    
    # Курсив
    cleaned = re.sub(r'\*(.*?)\*', r'\1', cleaned)  
    
    # Удаляем лишние пробелы и переносы строк
    cleaned = re.sub(r'\s+', ' ', cleaned)
    cleaned = cleaned.strip()

    return cleaned
    

In [96]:
c = get_chunks('./docs/deployment/')

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 167.22it/s]

Всего создано 96 чанков из 6 файлов


In [100]:
import pickle


In [3]:
client = QdrantClient('http://localhost:6333')
client.collection_exists(collection_name="{collection_name}")

False

In [4]:
if not client.collection_exists(collection_name="{collection_name}"):
    print('ff')

ff


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client import models
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Tuple
from md_parser import get_chunks
import pickle
import os
from tqdm import tqdm

class VectorDB:
    def __init__(self, collection_name: str = 'fastapi_docs'):
        self.collection_name = collection_name
        self.client = QdrantClient('http://localhost:6333') # Подключение к запущенному контейнеру
        self.embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
        # 'qilowoq/paraphrase-multilingual-mpnet-base-v2-en-ru'
        # self.embedding_model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

        if self.client.collection_exists(collection_name):
            print(f'Коллекция {collection_name} уже существует')
        else:
            self.client.create_collection(
                collection_name=collection_name,
                vectors_config= models.VectorParams(
                    size=768, # Размерность модели
                    distance=models.Distance.COSINE
                )
            )
            print(f'Коллекция {self.collection_name} создана')
        
    def add_chunks(self, chunks: List[Tuple[str, str]]):
        
        if os.path.isfile('points.pkl'):
            pass
                
        else:
            print(f'Добавление точек в {self.collection_name}')
            points = []
            for idx, (header, content) in tqdm(enumerate(chunks)):
                embedding = self.embedding_model.encode(content).tolist()
                point = models.PointStruct(
                    id=idx,
                    vector=embedding,
                    payload={
                        'content': content,
                        'header': header,
                        'source': self.collection_name,
                        'text_lenght': len(content)
                    }
                )
                points.append(point)
            with open('points.pkl', 'wb') as f:
                pickle.dump(points, f)

            self.client.upsert(
                collection_name=self.collection_name,
                points=points
            )



    def search(self, query:str, limit=5):
        '''Поиск похожих чанков'''
        query_embedding = self.embedding_model.encode(query).tolist()

        results = self.client.query_points(
            collection_name=self.collection_name,
            query=query_embedding,
            limit=limit,
            with_vectors=False
        )

        return results
            
            
        
if __name__ == '__main__':

    client = VectorDB()

    if os.path.isfile('chunks.pkl'):
        with open('chunks.pkl', 'rb') as f:
            chunks = pickle.load(f)
            print(f'Чанки загружены из файла {f.name}')
    else:
        chunks = get_chunks()
    client.add_chunks(chunks)
    while True:
        print(f'Для выхода введите "выход"')
        query = str(input('Введите вопрос: ')).lower().strip()
        if query =='выход':
            break
        limit = int(input('Введите лимит: '))
        if not query:
            print(f'Введите вопрос')
            continue

        result = client.search(query, limit)


    for result in result.points:
        print(f"Score: {result.score:.3f}")
        print(f"Header: {result.payload['header']}")
        print(f"Content: {result.payload['content'][:200]}...")
        print("---")

                



Коллекция fastapi_docs уже существует
Чанки загружены из файла chunks.pkl
Для выхода введите "выход"


In [ ]:
client.

In [57]:
for result in a.points[2:4]:
    print(result)
    print(f"Score: {result.score:.3f}")
    print(f"Header: {result.payload['header']}")
    print(f"Content: {result.payload['content'][:200]}...")
    print("---")

id=230 version=3 score=0.5712925 payload={'content': 'Создать Docker-образ Теперь, когда все файлы на своих местах, давайте создадим образ контейнера. * Перейдите в директорию проекта (в ту, где расположены `Dockerfile` и папка `app` с приложением). * Создай образ приложения FastAPI: ```console $ docker build -t myimage . ---> 100% ```', 'header': 'FastAPI и Docker-контейнеры', 'source': 'fastapi_docs', 'text_lenght': 283} vector=None shard_key=None order_value=None
Score: 0.571
Header: FastAPI и Docker-контейнеры
Content: Создать Docker-образ Теперь, когда все файлы на своих местах, давайте создадим образ контейнера. * Перейдите в директорию проекта (в ту, где расположены `Dockerfile` и папка `app` с приложением). * Со...
---
id=224 version=3 score=0.53488636 payload={'content': 'Образы контейнеров Docker является одним из основных инструментов для создания образов и контейнеров и управления ими. Существует общедоступный Docker Hub (https://hub.docker.com/) с подготовленными официальн

In [2]:
from gigachat import GigaChat
from vector_db import VectorDB
from dotenv import load_dotenv
import os
load_dotenv()
from dotenv import load_dotenv
API_KEY = os.getenv('GIGACHAT_API_KEY')

In [32]:
class RAGSystem:
    def __init__(self):
        self.vector_db = VectorDB()
        self.giga_client = GigaChat(
            credentials=API_KEY,
            verify_ssl_certs=False
        )
    def ask(self, question: str):
        # 1. Ищем релевантные чанки
        results = self.vector_db.search(question, limit=6)
        # 2. Формируем контекст
        context = '\n\n'.join([
            f'ИСточник: {r.payload['header']}\n{r.payload['content']}'
            for r in results.points
        ])
        # 3. Промт для GigaChat
        prompt = f"""
        Ответь на вопрос пользователя, основываясь на  предоставленный контекст.
        Если ответа нет в контексте, ответь своими словами или скажи "Не могу найти информацию в документации".

        Контекст:
        {context}

        Вопрос: {question}
        Ответ:
        """
        # 4. Ответ GigaChat
        response = self.giga_client.chat(prompt)
        return response.choices[0].message.content



In [33]:
# Использование
rag = RAGSystem()
answer = rag.ask("расскажи про машинное обучение")
print(answer)

Коллекция fastapi_docs уже существует
Не могу найти информацию в документации конкретно про машинное обучение в указанном контексте.

Однако расскажу кратко сам.

**Машинное обучение (МО)** — область информатики и математики, направленная на создание алгоритмов, позволяющих компьютерам обучаться на примерах данных и самостоятельно улучшать свою работу без явной программы действий программиста. 

Наиболее распространены следующие подходы МО:
- **Обучение с учителем**: алгоритм получает размеченные данные (где указаны правильные ответы), учится находить закономерности и предсказывать результаты новых примеров.
- **Обучение без учителя**: алгоритм находит скрытые структуры и взаимосвязи в неразмеченных данных.
- **Глубокое обучение (нейронные сети)**: использование многослойных искусственных нейронных сетей для решения сложных задач, включая распознавание изображений, речи, перевод текста и многое другое.

FastAPI часто используется разработчиками машинного обучения для быстрой и удобной 

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# Инициализация модели
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Вызов модели с сообщением
messages = [HumanMessage(content="Расскажи анекдот про программистов")]
response = chat_model.predict_messages(messages)
print(response.content)

/tmp/ipykernel_8349/1504278901.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)


ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_name': 'gpt-3.5-t...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error